In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import chain
from random import sample
import sklearn.model_selection as skl

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, plot_precision_recall_curve, f1_score, confusion_matrix
from skimage.transform import resize
##Import any other stats/DL/ML packages you may need here. E.g. Keras, scikit-learn, etc.

Using TensorFlow backend.


In [2]:
def load_pretrained_model():
    
    vgg_model = VGG16(include_top=False, weights='imagenet')
    return vgg_model


In [3]:
def build_my_model_v2():
    
    my_model_v2 = Sequential()
    
    my_model_v2.add(vgg_model)
    
    my_model_v2.add(GlobalAveragePooling2D())
    
    my_model_v2.add(Dense(256, activation='relu'))
    
    my_model_v2.add(BatchNormalization())
    
    my_model_v2.add(Dropout(0.5))
    
    my_model_v2.add(Dense(64, activation='relu'))
    
    my_model_v2.add(BatchNormalization())
    
    my_model_v2.add(Dropout(0.5))
    
    my_model_v2.add(Dense(32, activation='relu'))
    
    my_model_v2.add(BatchNormalization())
    
    my_model_v2.add(Dropout(0.5))
    
    my_model_v2.add(Dense(1, activation='sigmoid'))
    
    return my_model_v2

In [4]:
vgg_model = load_pretrained_model()
vgg_model.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [5]:
for layer in vgg_model.layers[0:17]:
    layer.trainable = False

In [6]:
for layer in vgg_model.layers:
    print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 True
block5_pool True


In [7]:
def check_dicom(filename): 
        
    print('Loading image file {} ......'.format(filename))
    ds = pydicom.dcmread(filename)   
    
    if ((ds.BodyPartExamined == 'CHEST') and (ds.PatientPosition in ['AP','PA']) and (ds.Modality in ['DX','CT'])):
        img = ds.pixel_array
        study_desc = ds.StudyDescription
        bodypartexamined = ds.BodyPartExamined
        patpos = ds.PatientPosition
        mod = ds.Modality
        return img, study_desc,bodypartexamined,patpos,mod
    
    study_desc = ds.StudyDescription
    bodypartexamined = ds.BodyPartExamined
    patpos = ds.PatientPosition
    mod = ds.Modality
    print("Image is not eligible for further processing since the Patient Position or Body Part Examined or Modality values are not qualified")
    print("Study Description:", study_desc)
    print("Body Part Examined", bodypartexamined)
    print("Patient Position", patpos)
    print("Modality", mod)
    print("\n")
    return None, study_desc,bodypartexamined,patpos,mod


In [8]:
def preprocess_image(img, img_mean, img_std, img_size): 
    
    img = img/ 255.0
    process_img = (img - img_mean / img_std)
    img_resize = resize(process_img, (img_size[1], img_size[2]))
    return np.resize(process_img, img_size)

In [9]:
def load_model(model_path):
    # todo
    vgg_model = build_my_model_v2()
#     vgg_model.load_weights(model_path)
    
    return vgg_model

In [10]:
def predict_image(model, img, thresh): 
    
    predict_model = model(img)
    
    if(predict_model > thresh):
        predict_model = 'Tested Pneumonia Positive'
    else:
        predict_model = "Tested Pneumonia Negative"
    return predict_model 

In [11]:
testdicoms = glob("*.dcm")

model_path = r'xray_class_my_model.h5'
weight_path = "{}_my_model.h5".format('xray_class')
# my_model_v2.load_weights(weight_path)


IMG_SIZE=(1,224,224,3)
img_std = 1
img_mean = 0
train_model = load_model(model_path)
thresh = 0.57

# use the .dcm files to test your prediction
for i in testdicoms:
    
    img = np.array([])
    img, study_desc,bodypartexamined,patpos,mod = check_dicom(i)
    
    if img is None:
        continue
    
    image_to_process = preprocess_image(img, img_mean, img_std, IMG_SIZE)
    predict_pneumonia = predict_image(train_model,image_to_process,thresh)
    print(predict_pneumonia)
    print("Study Description:", study_desc)
    print("Body Part Examined", bodypartexamined)
    print("Patient Position", patpos)
    print("Modality", mod)
    print("\n")
    

Loading image file test5.dcm ......

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Tested Pneumonia Negative
Study Description: No Finding
Body Part Examined CHEST
Patient Position PA
Modality CT


Loading image file test4.dcm ......
Image is not eligible for further processing since the Patient Position or Body Part Examined or Modality values are not qualified
Study Description: No Finding
Body Part Examined RIBCAGE
Patient Position PA
Modality DX


Loading image file test6.dcm ......
Image is not eligible for further processing since the Patient 